In [1]:
import tensorflow as tf 
import numpy as np
import keras
import keras.backend as K 
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, BatchNormalization
from keras.layers import Conv2D, GlobalMaxPooling1D, MaxPooling2D
from keras.optimizers import Adam
from keras.utils.training_utils import multi_gpu_model

Using TensorFlow backend.


In [2]:
import numpy
import csv
import h5py
import gc 
import argparse
import os 
import psutil

In [3]:
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1"

gpu_options = tf.GPUOptions(allow_growth=True)

sess = tf.Session(config=tf.ConfigProto(gpu_options=gpu_options))

In [4]:
n_hidden_units = 50
n_convpool_layers = 1
n_convlayers = 2
n_reglayers = 2
max_poolsize = (2,2)
train_set_ratio = 0.7
valid_set_ratio = 0.15
set_split_ratio34 = 0.5
drop_rate = 0.0
n_filters = 130 #130 before 
reg_factor = 0.0
kernel_size = (2,2)
input_strides = 1
kernel_init = 'uniform'
cost_function = 'mean_squared_error'
batch_size = 64#128
feed_batch_size = 256
input_sample_shape = [128,128,5]
#target_sample_shape = [50]
n_epochs = 35
n_output = [0,50]
n_set1 = 10000
n_set2 = None
n_set100 = 10000 
n_post_test = 5000
xnorm_axis = (0,1,2,3)
early_stop_delta = 0.01 # 0.01 change or above is considered improvement
early_stop_patience = 10 


In [27]:
def get_x_mean_n_range():
    ds = ds = h5py.File("../data/data2D%d.h5","r",driver='family', memb_size=2500*10**6)
    x_min = np.zeros((1,5),dtype = ds["random"]["x0"].dtype)
    x_max = np.zeros((1,5),dtype = ds["random"]["x0"].dtype)
    x_mean = np.zeros((1,5),dtype = ds["random"]["x0"].dtype)
    count = 0
    for key1 in list(ds.keys()):
        for key2 in ds[key1].keys():
            if key2.find("x")>=0 :
                temp_x = ds[key1][key2][:]
                if temp_x.shape[0] != 64:
                    continue
                    
                #handle mean
                count+=1
                x_mean += np.mean(temp_x,axis=(0,1,2))
                
                #handle max
                temp_max = np.amax(temp_x,axis=(0,1,2))
                temp_batch = np.vstack([temp_max,x_max])
                x_max = np.amax(temp_batch,axis=(0))
                
                #handle min
                temp_min = np.amin(temp_x,axis =(0,1,2))
                temp_batch = np.vstack([temp_min,x_min])
                x_min = np.amin(temp_batch,axis=(0))
                
    x_mean = x_mean/count     
    x_range = x_max-x_min
    return x_mean,x_range

def get_y_mean_n_range():
    ds = ds = h5py.File("../data/data2D%d.h5","r",driver='family', memb_size=2500*10**6)
    y_min = 0 
    y_max = 0
    y_mean = 0
    count = 0
    for key1 in list(ds.keys()):
        for key2 in ds[key1].keys():
            if key2.find("y")>=0 :
                temp_y = np.real(ds[key1][key2][:])
                if temp_y.shape[0] != 64:
                    continue
                    
                #handle mean
                count+=1
                y_mean += np.mean(temp_y)
                
                #handle max
                temp_max = np.max(temp_y)
                y_max = temp_max if temp_max>y_max else y_max
                
                #handle min
                temp_min = np.min(temp_y)
                y_min = temp_min if temp_min<y_min else y_min
                
    y_mean = y_mean/count     
    y_range = y_max-y_min
    return y_mean,y_range
                
def generate_data_from_file(x_mean,x_range,y_mean,y_range):
    while 1:
        ds = h5py.File("../data/data2D%d.h5","r",driver='family', memb_size=2500*10**6)
        for key1 in list(ds.keys()):
            x_count = 0
            y_count = 0
            for key2 in list(ds[key1].keys()):   #0-399 or 0-199
                if key2[0]!="x" or ds[key1][key2].shape[0]>64:
                    continue
                
                x_raw = ds[key1][key2][:]
                y_key = "y"+key2[1:]
                y_raw = ds[key1][y_key][:]
#                 x_mean = np.mean(x_raw,axis=(0,1,2))
#                 x_max = np.amax(x_raw,axis=(0,1,2))
                x_normed = (x_raw-x_mean)/x_range
                
#                 y_mean = np.mean(y_raw,axis=())
#                 y_max = np.max(y_raw,axis=())
#                 y_min = np.min(y_raw,axis=())
                y_normed = (y_raw-y_mean)/y_range
                
                yield(x_normed,y_normed)

def get_validation_data(x_mean,x_range,y_mean,y_range):
    ds = h5py.File("../data/data2D%d.h5","r",driver='family', memb_size=2500*10**6)
    x_val_raw = ds["random"]["x0"][:]
    y_val_raw = ds["random"]["y0"][:]
    
#     x_val_mean = np.mean(x_val_raw,axis=(0,1,2))
#     x_val_max = np.amax(x_val_raw,axis=(0,1,2))
    x_val_normed = (x_val_raw-x_mean)/x_range
    
#     y_val_mean = np.mean(y_val_raw,axis=())
#     y_val_max = np.max(y_val_raw,axis=())
#     y_val_min = np.min(y_val_raw,axis=())
    y_val_normed = (y_val_raw-y_mean)/y_range
    
    return x_val_normed,y_val_normed

def get_test_data(x_mean,x_range,y_mean,y_range):
    x_test_raw = np.load("../data/inputMatrix.npz")["inputMatrix"][:]
    y_test_raw = np.load("../data/Solution.npz")["frequency1"][:,:50]
    x_test = (x_test_raw-x_mean)/x_range
    y_test = (y_test_raw-y_mean)/y_range
    return x_test,y_test,x_test_raw,y_test_raw
    


In [6]:
x_mean,x_range = get_x_mean_n_range()
print(x_mean)
print(x_range)

[[1.49880835e+11 4.40176392e+03 2.74842257e-01 2.49137578e-01
  2.49167977e-01]]
[2.99999999e+11 8.00000000e+03 4.49999999e-01 4.99999586e-01
 4.99985816e-01]


In [7]:
y_mean,y_range = get_y_mean_n_range()
print(y_mean)
print(y_range)

1096210.1699824824
6727841.0


In [9]:
def create_model():
    model=Sequential()

    model.add(Conv2D(filters=n_filters, kernel_size = kernel_size ,strides= input_strides,
                     input_shape=(128,128,5),kernel_initializer= kernel_init))
    model.add(Activation('relu'))

    #model.add(Conv2D(filters=n_filters, kernel_size = kernel_size ,strides= input_strides,
    #                 kernel_initializer= kernel_init))
    #model.add(Activation('relu'))

    #model.add(Conv2D(filters=n_filters, kernel_size = kernel_size ,strides= input_strides,
    #                 kernel_initializer= kernel_init))
    #model.add(Activation('relu'))


    model.add(MaxPooling2D(pool_size=max_poolsize))

    #model.add(BatchNormalization())

    model.add(Conv2D(filters=n_filters, kernel_size = kernel_size ,strides= input_strides,
                     kernel_initializer= kernel_init))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=max_poolsize))

    #model.add(BatchNormalization())

    model.add(Conv2D(filters=n_filters, kernel_size = kernel_size ,strides= input_strides,
                     kernel_initializer= kernel_init))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=max_poolsize))
    
#     model.add(Conv2D(filters=n_filters, kernel_size = kernel_size ,strides= input_strides,
#                      kernel_initializer= kernel_init))
#     model.add(Activation('relu'))
    #model.add(MaxPooling2D(pool_size=max_poolsize))

    model.add(Flatten())
    model.add(Dense(n_hidden_units))
    model.add(Activation('relu'))
    model.add(Dense(n_output[1]-n_output[0], activation='linear'))

    earlyStopping = keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=early_stop_delta, patience=early_stop_patience, verbose=2, mode='auto')

    model.summary()
    return model

In [24]:
model = create_model()
parallel_model = multi_gpu_model(model, gpus=2)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_7 (Conv2D)            (None, 127, 127, 130)     2730      
_________________________________________________________________
activation_9 (Activation)    (None, 127, 127, 130)     0         
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 63, 63, 130)       0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 62, 62, 130)       67730     
_________________________________________________________________
activation_10 (Activation)   (None, 62, 62, 130)       0         
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 31, 31, 130)       0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 30, 30, 130)       67730     
__________

In [25]:
#model.compile(loss=cost_function, optimizer='adam', metrics=["MAE"])
parallel_model.compile(loss=cost_function, optimizer='adam', metrics=["MAE"])

In [12]:
x_val,y_val = get_validation_data(x_mean,x_range,y_mean,y_range)
print(x_val.shape,y_val.shape)
print(x_val[0][0][0])
print(y_val[0])

(15360, 128, 128, 5) (15360, 50)
[-0.41074545  0.38468491 -0.3320371  -0.10327968  0.22063786]
[-0.13456129-2.45686761e-12j -0.12932207+4.80859536e-12j
 -0.12199552+8.19367848e-13j -0.11168992-1.68551249e-11j
 -0.10281598+4.14451841e-11j -0.10213102-1.49313461e-11j
 -0.09387112-2.45010227e-11j -0.09083606+1.40311443e-11j
 -0.08885166+3.80655707e-12j -0.08095209-2.16298212e-11j
 -0.07790928+1.09215900e-11j -0.07704957+3.91925346e-11j
 -0.07190627+4.68126753e-11j -0.06895456-3.24584838e-11j
 -0.06569687-4.26053290e-11j -0.05268907+3.67771889e-11j
 -0.04989202+1.07599957e-10j -0.04768465-1.02832631e-10j
 -0.04347732+5.24321177e-11j -0.04247696+9.31105193e-11j
 -0.03967341-1.12276397e-10j -0.03773416+8.79517709e-11j
 -0.03653685-8.02528877e-11j -0.03505816+1.42429221e-10j
 -0.03318293+1.50398402e-10j -0.02809242-5.27961702e-11j
 -0.02458743-3.43634808e-11j -0.02283694-2.49335951e-11j
 -0.02070505+2.80978879e-10j -0.01989186+1.10593229e-10j
 -0.01703646+4.94370690e-11j -0.01440656+6.2826494

In [28]:
parallel_model.fit_generator(generate_data_from_file(x_mean,x_range,y_mean,y_range),
                    steps_per_epoch=999,epochs=12,validation_data=(x_val,y_val))

Epoch 1/12


/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/numpy/core/numeric.py:538: ComplexWarning: Casting complex values to real discards the imaginary part
  return array(a, dtype, copy=False, order=order)


999/999 [==============================] - 415s 415ms/step - loss: 4.4319e-04 - mean_absolute_error: 0.0105 - val_loss: 1.9406e-04 - val_mean_absolute_error: 0.0112
Epoch 2/12
999/999 [==============================] - 361s 361ms/step - loss: 1.0335e-04 - mean_absolute_error: 0.0063 - val_loss: 9.2225e-04 - val_mean_absolute_error: 0.0282
Epoch 3/12
999/999 [==============================] - 362s 362ms/step - loss: 7.9816e-05 - mean_absolute_error: 0.0058 - val_loss: 0.0012 - val_mean_absolute_error: 0.0318
Epoch 4/12
999/999 [==============================] - 361s 361ms/step - loss: 8.9057e-05 - mean_absolute_error: 0.0059 - val_loss: 0.0012 - val_mean_absolute_error: 0.0317
Epoch 5/12
999/999 [==============================] - 361s 361ms/step - loss: 7.9172e-05 - mean_absolute_error: 0.0056 - val_loss: 8.8596e-04 - val_mean_absolute_error: 0.0272
Epoch 6/12
999/999 [==============================] - 360s 361ms/step - loss: 7.0046e-05 - mean_absolute_error: 0.0054 - val_loss: 8.3828e-

In [29]:
model.save("model_wholeSet_normed_c3_f130_e12.h5")

In [49]:
x_test,y_test,x_test_raw,y_test_raw = get_test_data(x_mean,x_range,y_mean,y_range)

In [50]:
loss,mae=model.evaluate(x_test,y_test)
print("loss:",loss)
print("mae:",mae)

22/22 [==============================] - 0s 3ms/step
loss: 0.0013233348727226257
mae: 0.03408118709921837
